In [6]:
import random

import boto3
import polars as pl
import numpy as np

from typing import List, Any
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from utils.io.s3 import DataFrameType, download_dataframe, upload_dataframe

import constants

In [2]:
s3_session = boto3.session.Session()
s3_client = s3_session.client(service_name='s3', endpoint_url='https://storage.yandexcloud.net')

In [4]:
ad_features = download_dataframe(s3_client, constants.S3_BUCKET, constants.DATA_PATH / 'ad_features.parquet', DataFrameType.POLARS)
user_features = download_dataframe(s3_client, constants.S3_BUCKET, constants.DATA_PATH / 'user_features.parquet', DataFrameType.POLARS)
behavioral_logs = download_dataframe(s3_client, constants.S3_BUCKET, constants.DATA_PATH / 'behavior_logs.parquet', DataFrameType.POLARS)
data = download_dataframe(s3_client, constants.S3_BUCKET, constants.DATA_PATH / 'train.parquet', DataFrameType.POLARS)
test = download_dataframe(s3_client, constants.S3_BUCKET, constants.DATA_PATH / 'test.parquet', DataFrameType.POLARS)

## Описание данных

### ad_features

| Поле            | Тип                  | Описание                                 |
|---              |---                   |---                                       |
| adgroup_id      | int                  | Идентификатор рекламы                    |
| cate_id         | int                  | Идентификатор категории                  |
| campaign_id     | int                  | Идентификатор рекламной компании         |
| customer        | int                  | Идентификатор рекламодателя              |
| brand           | str                  | Идентификатор бренда                     |
| price           | int                  | Цена товара                              |

Рекламное объявление принадлежит одному товару, при этом у товара есть бренд и категория


### user_features

| Поле            | Тип                  | Описание                                 |
|---              |---                   |---                                       |
| userid          | int                  | Идентификатор пользователя               |
| cms_segid       | int                  | Идентификатор сегментированной группы пользователей|
| cms_group_id    | int                  | Идентификатор группы пользователей       |
| final_gender_code | int                | Пол пользователя (мужской – 1, женский – 2)|
| age_level       | int                  | Категория возраста пользователя          |
| pvalue_level    | int                  | Уровень потребления                      |
| shopping_level  | int                  | Уровень вовлеченности (значения из диапазона 1-3 от слабой до сильной) |
| occupation      | int                  | Род занятости (студент или нет)          |
| new_user_class_level | int             | Населенность города проживания           |

### behavioral_logs

| Поле            | Тип                  | Описание                                 |
|---              |---                   |---                                       |
| user            | int                  | Идентификатор пользователя               |
| time_stamp      | int                  | Время                                    |
| btag            | str                  | Тип действия (**pv**: просмотр, **cart**: добавление в корзину, **fav**: добавление в избранное, **buy**: покупка) |
| cate            | int                  | Идентификатор категории                  |
| brand           | str                  | Идентификатор бренда                     |


### data

| Поле            | Тип                  | Описание                                 |
|---              |---                   |---                                       |
| user            | int                  | Идентификатор пользователя               |
| time_stamp      | int                  | Время                                    |
| adgroup_id      | int                  | Идентификатор рекламы                    |
| clk             | int                  | Был ли клик?                             |

## Оценивание

В качестве метрики качества используется ndcg@10. Чтобы получить максимальный балл, достаточно добиться ndcg@10 = 0.03

В качестве `y_relevant` используется те рекламные объявления, по которым были клики после собранной истории взаимодействий

In [5]:
TOP_K = 10


def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))

def user_recall(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of found relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / len(set(y_rel))


def user_ndcg(y_rel: List[Any], y_rec: List[Any], k: int = TOP_K) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: ndcg metric for user recommendations
    """
    dcg = sum([1. / np.log2(idx + 2) for idx, item in enumerate(y_rec[:k]) if item in y_rel])
    idcg = sum([1. / np.log2(idx + 2) for idx, _ in enumerate(zip(y_rel, np.arange(k)))])
    return dcg / idcg

## Бейзлайн с использованием behavioral logs

В качестве простого бейзлайна для каждого пользователя найдем категорию рекламных объявлений, наиболее интересную ему. Затем для каждой категории найдем самые популярные по метрике `ctr` (Click-through rate).

In [7]:
data = data.sort('time_stamp')

timestamp_threshold = data['time_stamp'].quantile(0.9)
train_df = data.filter(pl.col('time_stamp') <= timestamp_threshold)
test_df = data.filter(pl.col('time_stamp') > timestamp_threshold)

In [8]:
# веса примерно соответствуют behavioral_logs['btag'].value_counts()
weight_dict = {
    'pv': 1,
    'cart': 40,
    'fav': 80,
    'buy': 80
}

behavioral_logs_top_categories = (
    behavioral_logs
    # для валидации отфильтруем все события как в тренировочной выборке
    .filter(pl.col('time_stamp') <= timestamp_threshold)
     # каждое событие преобразуем в вес
    .with_columns([pl.col('btag').apply(weight_dict.get).alias('weight')])
    .groupby('user', 'cate')
    # итоговый вес для категории – сумма весов событий
    .agg(pl.col('weight').sum())
    # для каждого пользователя оставим категорию с максимальным весом
    .sort(['user', 'weight'], descending=True)
    .unique('user', keep='first')
)
behavioral_logs_top_categories

user,cate,weight
i64,i64,i64
1141729,4283,166
1141726,4505,5
1141725,10905,93
1141724,4385,135
1141723,6421,14
1141722,4263,80
1141721,4282,1
1141720,4520,51
1141718,6432,60


In [9]:
def get_top_candidates(x: List[dict]) -> List[int]:
    """
    Для списка структур с полями ctr и adgroup_id сортирует его по убыванию ctr и возвращает
    TOP_K соответствующих идентификаторов adgroup_id
    """
    x = sorted(x, key=lambda v: -v['ctr'])[:TOP_K]
    return list(map(lambda x: x['adgroup_id'], x))

grouped_tops = (
    train_df
    .join(ad_features, on='adgroup_id')
    # для пары cate_id и adgroup_id вычисляем ctr
    .groupby('cate_id', 'adgroup_id')
    .agg([
        pl.col('clk').mean().alias('ctr'),
        pl.count().alias('count')
    ])
    # уберем те рекламные объявления, которые были показаны всего N раз
    .filter(pl.col('count') > 1)
    # сгруппируем ctr и adgroup_id в словарь и вызовем функцию get_top_candidates
    .with_columns([pl.struct(ctr=pl.col('ctr'), adgroup_id=pl.col('adgroup_id')).alias('struct')])
    .groupby('cate_id')
    .agg(pl.col('struct').apply(get_top_candidates).alias('y_rec'))
)
grouped_tops

cate_id,y_rec
i64,list[i64]
8064,"[351582, 44232, … 339624]"
6720,"[7953, 404192, … 268161]"
6624,"[531599, 286359]"
8960,"[70283, 513966, … 171830]"
11904,"[125464, 49320, … 86975]"
8416,"[302623, 74549]"
2208,"[442638, 351878, … 229901]"
6368,"[363814, 138620, … 775932]"
6912,"[492814, 736966, … 94817]"


In [10]:
recs = (
    test
    .join(behavioral_logs_top_categories, left_on='user_id', right_on='user')
    .join(grouped_tops, left_on='cate', right_on='cate_id')
    .select('user_id', 'y_rec')
)
recs

user_id,y_rec
i64,list[i64]
1141720,"[511663, 692584, … 348929]"
1141714,"[725297, 547310, … 462183]"
1141710,"[196526, 141797, … 151029]"
1141709,"[518334, 595289, … 520157]"
1141708,"[577671, 741993, … 441383]"
1141707,"[404677, 270646, … 458672]"
1141706,"[7304, 204588, … 703046]"
1141705,"[577671, 741993, … 441383]"
1141697,"[105110, 440179, … 630699]"


In [11]:
test_grouped_df = (
    test_df
    # релевантны только те объекты, для которых был клик
    .filter(pl.col('clk') == 1)
    .groupby('user')
    .agg(pl.col('adgroup_id').alias('y_rel'))
)

ndcg_list = []
recall_list = []
for _, y_rel, y_rec in test_grouped_df.join(recs, left_on='user', right_on='user_id').rows():
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    recall_list.append(user_recall(y_rel, y_rec))
    
mean_ndcg = np.mean(ndcg_list)
mean_recall = np.mean(recall_list)
print(f'NDCG@{TOP_K} = {mean_ndcg:.4f}, Recall@{TOP_K} = {mean_recall:.4f}')

NDCG@10 = 0.0006, Recall@10 = 0.0014


Построим рекомендации по всем данным

In [12]:
behavioral_logs_top_categories = (
    behavioral_logs
     # каждое событие преобразуем в вес
    .with_columns([pl.col('btag').apply(weight_dict.get).alias('weight')])
    .groupby('user', 'cate')
    # итоговый вес для категории – сумма весов событий
    .agg(pl.col('weight').sum())
    # для каждого пользователя оставим категорию с максимальным весом
    .sort(['user', 'weight'], descending=True)
    .unique('user', keep='first')
)
behavioral_logs_top_categories

user,cate,weight
i64,i64,i64
1141729,4283,305
1141726,4505,5
1141725,10905,93
1141724,4385,135
1141723,6421,14
1141722,4263,80
1141721,6426,4
1141720,4520,51
1141718,6432,60


In [13]:
grouped_tops = (
    data
    .join(ad_features, on='adgroup_id')
    # для пары cate_id и adgroup_id вычисляем ctr
    .groupby('cate_id', 'adgroup_id')
    .agg([
        pl.col('clk').mean().alias('ctr'),
        pl.count().alias('count')
    ])
    # уберем те рекламные объявления, которые были показаны всего N раз
    .filter(pl.col('count') > 1)
    # сгруппируем ctr и adgroup_id в словарь и вызовем функцию get_top_candidates
    .with_columns([pl.struct(ctr=pl.col('ctr'), adgroup_id=pl.col('adgroup_id')).alias('struct')])
    .groupby('cate_id')
    .agg(pl.col('struct').apply(get_top_candidates).alias('y_rec'))
)
grouped_tops

cate_id,y_rec
i64,list[i64]
10112,"[27339, 18836, … 97599]"
7520,"[84543, 256698, … 165374]"
800,"[14056, 177207, … 708371]"
11136,"[335660, 449081, … 57849]"
7360,"[195370, 195342]"
9984,"[165761, 221503, … 345683]"
4544,"[56302, 284536, … 132707]"
10560,"[292218, 62933, … 358535]"
9376,"[130887, 282312, … 406208]"


In [14]:
(
    test
    .join(behavioral_logs_top_categories, left_on='user_id', right_on='user')
    .join(grouped_tops, left_on='cate', right_on='cate_id')
    .select('user_id', 'y_rec')
    .write_parquet('submission.parquet')
)